In [24]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
import copy
import scipy

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset, Dataset


import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor

from image_list import ImageList


import utils

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def test_model_10l_all_datasets(model):
    #Mean: tensor([0 1309, 0.1309, 0.1309])
    #Standard Deviation: tensor([0.2893, 0.2893, 0.2893])
    test_dataset_mnist = datasets.MNIST(
        root = 'data', 
        train = False, 
        transform = transforms.Compose([transforms.Grayscale(3),
                                        transforms.Resize((32, 32)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ]),
    )
    
    #Mean: tensor([0.4377, 0.4438, 0.4728])
    #Standard Deviation: tensor([0.1980, 0.2010, 0.1970])
    test_dataset_svhn = datasets.SVHN(
        root = 'data/SVHN', 
        split = 'test', 
        transform = transforms.Compose([transforms.Resize((32, 32)),
                                        ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ]),
        download = True,
    )
    
    ## Creating a Concatened Dataset
    test_dataset_combined = ConcatDataset([test_dataset_mnist,test_dataset_svhn])

    # dataloaders
    test_dataloader_mnist = torch.utils.data.DataLoader(test_dataset_mnist,batch_size=64, shuffle=False)
    test_dataloader_svhn = torch.utils.data.DataLoader(test_dataset_svhn,batch_size=64, shuffle=False)
    # test_dataloader_combined = DataLoader(test_dataset_combined, batch_size=64, shuffle=False)
    
    test_loss, accuracy = utils.evaluate(model, test_dataloader_mnist)
    print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")
    combined_accuracy = accuracy * 10000
    combined_test_loss = test_loss * 10000
    test_loss, accuracy = utils.evaluate(model, test_dataloader_svhn)
    print(f"model_copycat in svhn dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")
    combined_accuracy+= accuracy * 26032
    combined_test_loss+= test_loss * 26032
    
    combined_accuracy = combined_accuracy/(10000 + 26032)
    combined_test_loss = combined_test_loss/(10000 + 26032)
    # test_loss, accuracy = utils.evaluate(model, test_dataloader_combined)
    print(f"model_copycat in combined dataset- Test Loss: {combined_test_loss:.4f}, Accuracy: {combined_accuracy:.2f}")


# 10 outputs

## Original Models

In [7]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/original/mnist_32.pth'))

model_copycat in mnist dataset- Test Loss: 0.0295, Accuracy: 99.29
model_copycat in svhn dataset- Test Loss: 5.5442, Accuracy: 22.36
model_copycat in combined dataset- Test Loss: 4.0137, Accuracy: 43.71


In [8]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/original/svhn_32.pth'))

model_copycat in mnist dataset- Test Loss: 1.5104, Accuracy: 65.01
model_copycat in svhn dataset- Test Loss: 0.2867, Accuracy: 93.10
model_copycat in combined dataset- Test Loss: 0.6263, Accuracy: 85.31


In [9]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/original/combined_32.pth'))

model_copycat in mnist dataset- Test Loss: 0.1971, Accuracy: 94.56
model_copycat in svhn dataset- Test Loss: 0.3046, Accuracy: 92.98
model_copycat in combined dataset- Test Loss: 0.2748, Accuracy: 93.42


## Hard Learning 

In [12]:
## Mnist
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/hard/copycat_m10.pth'))

model_copycat in mnist dataset- Test Loss: 0.0295, Accuracy: 99.29
model_copycat in svhn dataset- Test Loss: 5.5442, Accuracy: 22.36
model_copycat in combined dataset- Test Loss: 4.0137, Accuracy: 43.71


In [13]:
# Hard Learning ms
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/hard/copycat_m10_s10.pth'))

model_copycat in mnist dataset- Test Loss: 13.7482, Accuracy: 49.55
model_copycat in svhn dataset- Test Loss: 21.0525, Accuracy: 71.77
model_copycat in combined dataset- Test Loss: 19.0253, Accuracy: 65.60


In [14]:
# Hard Learning combined
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/hard/copycat_c10.pth'))

model_copycat in mnist dataset- Test Loss: 1.3398, Accuracy: 83.05
model_copycat in svhn dataset- Test Loss: 2.2311, Accuracy: 78.73
model_copycat in combined dataset- Test Loss: 1.9838, Accuracy: 79.93


## Soft Learning

### Mnist

In [6]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_m1_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.5161, Accuracy: 76.31
model_copycat in svhn dataset- Test Loss: 2.2311, Accuracy: 22.04
model_copycat in combined dataset- Test Loss: 2.0327, Accuracy: 37.10


In [16]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_m2_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.5251, Accuracy: 84.08
model_copycat in svhn dataset- Test Loss: 2.2289, Accuracy: 22.15
model_copycat in combined dataset- Test Loss: 2.0336, Accuracy: 39.33


In [17]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_m3_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.4312, Accuracy: 89.79
model_copycat in svhn dataset- Test Loss: 2.2285, Accuracy: 22.40
model_copycat in combined dataset- Test Loss: 2.0072, Accuracy: 41.10


In [18]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_m4_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.4559, Accuracy: 90.39
model_copycat in svhn dataset- Test Loss: 2.2292, Accuracy: 22.40
model_copycat in combined dataset- Test Loss: 2.0146, Accuracy: 41.27


In [11]:
model_copycat = utils.create_resnet_model('./pths/copycat_m5_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.4180, Accuracy: 97.52


In [14]:
model_copycat = utils.create_resnet_model('./pths/copycat_m6_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.3735, Accuracy: 96.39


In [15]:
model_copycat = utils.create_resnet_model('./pths/copycat_m7_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.3784, Accuracy: 98.65


In [16]:
model_copycat = utils.create_resnet_model('./pths/copycat_m8_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.3347, Accuracy: 98.76


In [17]:
model_copycat = utils.create_resnet_model('./pths/copycat_m9_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.2842, Accuracy: 98.75


In [18]:
model_copycat = utils.create_resnet_model('./pths/copycat_m10_soft.pth')
test_loss, accuracy = utils.evaluate(model_copycat, test_dataloader_mnist)
print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")

model_copycat in mnist dataset- Test Loss: 1.3474, Accuracy: 98.72


### MNIST -> SVHN

In [11]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s1_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8077, Accuracy: 71.08
model_copycat in svhn dataset- Test Loss: 1.6717, Accuracy: 88.58
model_copycat in combined dataset- Test Loss: 1.7095, Accuracy: 83.72


In [12]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s2_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8162, Accuracy: 71.01
model_copycat in svhn dataset- Test Loss: 1.6560, Accuracy: 89.80
model_copycat in combined dataset- Test Loss: 1.7004, Accuracy: 84.59


In [13]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s3_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8277, Accuracy: 69.57
model_copycat in svhn dataset- Test Loss: 1.6481, Accuracy: 90.04
model_copycat in combined dataset- Test Loss: 1.6979, Accuracy: 84.36


In [14]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s4_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8359, Accuracy: 68.48
model_copycat in svhn dataset- Test Loss: 1.6463, Accuracy: 90.56
model_copycat in combined dataset- Test Loss: 1.6989, Accuracy: 84.43


In [15]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s5_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8381, Accuracy: 66.79
model_copycat in svhn dataset- Test Loss: 1.6282, Accuracy: 90.50
model_copycat in combined dataset- Test Loss: 1.6865, Accuracy: 83.92


In [17]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s6_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8362, Accuracy: 66.21
model_copycat in svhn dataset- Test Loss: 1.6200, Accuracy: 90.44
model_copycat in combined dataset- Test Loss: 1.6800, Accuracy: 83.72


In [18]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s7_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8365, Accuracy: 66.47
model_copycat in svhn dataset- Test Loss: 1.6218, Accuracy: 90.65
model_copycat in combined dataset- Test Loss: 1.6814, Accuracy: 83.94


In [19]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s8_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8278, Accuracy: 67.75
model_copycat in svhn dataset- Test Loss: 1.6254, Accuracy: 90.80
model_copycat in combined dataset- Test Loss: 1.6815, Accuracy: 84.41


In [23]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s9_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8307, Accuracy: 65.96
model_copycat in svhn dataset- Test Loss: 1.6177, Accuracy: 90.62
model_copycat in combined dataset- Test Loss: 1.6768, Accuracy: 83.78


In [24]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s10_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8321, Accuracy: 67.15
model_copycat in svhn dataset- Test Loss: 1.6211, Accuracy: 90.76
model_copycat in combined dataset- Test Loss: 1.6797, Accuracy: 84.21


In [25]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_m8_s11_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8270, Accuracy: 67.83
model_copycat in svhn dataset- Test Loss: 1.6221, Accuracy: 90.87
model_copycat in combined dataset- Test Loss: 1.6790, Accuracy: 84.47


In [32]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_m8_s12_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.8242, Accuracy: 67.01
model_copycat in svhn dataset- Test Loss: 1.6116, Accuracy: 90.87
model_copycat in combined dataset- Test Loss: 1.6706, Accuracy: 84.25


### MNIST+SVHN

In [4]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms1_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6713, Accuracy: 86.78
model_copycat in svhn dataset- Test Loss: 1.9707, Accuracy: 44.95
model_copycat in combined dataset- Test Loss: 1.8876, Accuracy: 56.56


In [5]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms2_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6662, Accuracy: 89.68
model_copycat in svhn dataset- Test Loss: 1.9090, Accuracy: 54.28
model_copycat in combined dataset- Test Loss: 1.8416, Accuracy: 64.10


In [6]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms3_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6384, Accuracy: 89.82
model_copycat in svhn dataset- Test Loss: 1.8737, Accuracy: 53.39
model_copycat in combined dataset- Test Loss: 1.8084, Accuracy: 63.50


In [9]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms4_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.5787, Accuracy: 92.34
model_copycat in svhn dataset- Test Loss: 1.8681, Accuracy: 60.40
model_copycat in combined dataset- Test Loss: 1.7878, Accuracy: 69.27


In [10]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms5_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.5796, Accuracy: 89.82
model_copycat in svhn dataset- Test Loss: 1.8472, Accuracy: 67.18
model_copycat in combined dataset- Test Loss: 1.7730, Accuracy: 73.46


In [11]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms6_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6140, Accuracy: 90.22
model_copycat in svhn dataset- Test Loss: 1.8791, Accuracy: 39.50
model_copycat in combined dataset- Test Loss: 1.8055, Accuracy: 53.57


In [12]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms7_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6101, Accuracy: 88.06
model_copycat in svhn dataset- Test Loss: 1.8401, Accuracy: 66.88
model_copycat in combined dataset- Test Loss: 1.7763, Accuracy: 72.75


In [13]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms8_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6110, Accuracy: 92.11
model_copycat in svhn dataset- Test Loss: 1.8598, Accuracy: 56.08
model_copycat in combined dataset- Test Loss: 1.7907, Accuracy: 66.08


In [14]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/copycat_ms9_soft.pth'))

model_copycat in mnist dataset- Test Loss: 1.6265, Accuracy: 91.48
model_copycat in svhn dataset- Test Loss: 1.8705, Accuracy: 59.60
model_copycat in combined dataset- Test Loss: 1.8028, Accuracy: 68.44


In [40]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_9/soft/copycat_ms10_soft.pth'))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


### Extracted from Combined Model

# 20 outputs

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def test_model_20l_all_datasets(model):
    test_dataset_mnist = datasets.MNIST(
        root = 'data', 
        train = False, 
        transform = transforms.Compose([transforms.Grayscale(3),
                                        transforms.Resize((32, 32)),
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ]),
    )
    
    add_ten = lambda y: y + 10
    test_dataset_svhn = datasets.SVHN(
        root = 'data/SVHN', 
        split = 'test', 
        transform = transforms.Compose([transforms.Resize((32, 32)),
                                        ToTensor(),
                                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                       ]),
        target_transform =add_ten,
        download = True,
    )
    
    ## Creating a Concatened Dataset
    test_dataset_combined = ConcatDataset([test_dataset_mnist,test_dataset_svhn])

    # dataloaders
    test_dataloader_mnist = torch.utils.data.DataLoader(test_dataset_mnist,batch_size=64, shuffle=False)
    test_dataloader_svhn = torch.utils.data.DataLoader(test_dataset_svhn,batch_size=64, shuffle=False)
    # test_dataloader_combined = DataLoader(test_dataset_combined, batch_size=64, shuffle=False)
    
    test_loss, accuracy = utils.evaluate(model, test_dataloader_mnist)
    print(f"model_copycat in mnist dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")
    combined_accuracy = accuracy * 10000
    combined_test_loss = test_loss * 10000
    test_loss, accuracy = utils.evaluate(model, test_dataloader_svhn)
    print(f"model_copycat in svhn dataset- Test Loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}")
    combined_accuracy+= accuracy * 26032
    combined_test_loss+= test_loss * 26032
    
    combined_accuracy = combined_accuracy/(10000 + 26032)
    combined_test_loss = combined_test_loss/(10000 + 26032)
    # test_loss, accuracy = utils.evaluate(model, test_dataloader_combined)
    print(f"model_copycat in combined dataset- Test Loss: {combined_test_loss:.4f}, Accuracy: {combined_accuracy:.2f}")

## Hard train

In [41]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms1.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.9112, Accuracy: 10.63
model_copycat in svhn dataset- Test Loss: 2.2501, Accuracy: 24.72
model_copycat in combined dataset- Test Loss: 2.2501, Accuracy: 20.81


In [27]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms1.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.9112, Accuracy: 10.63
model_copycat in svhn dataset- Test Loss: 2.2501, Accuracy: 24.72
model_copycat in combined dataset- Test Loss: 3.2662, Accuracy: 20.81


In [28]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms1.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.9112, Accuracy: 10.63
model_copycat in svhn dataset- Test Loss: 16.9432, Accuracy: 14.01
model_copycat in combined dataset- Test Loss: 13.8815, Accuracy: 13.07


In [21]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms6.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.0643, Accuracy: 27.16
model_copycat in svhn dataset- Test Loss: 2.0463, Accuracy: 12.27
model_copycat in combined dataset- Test Loss: 2.8839, Accuracy: 16.40


In [23]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms7.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.1644, Accuracy: 31.22
model_copycat in svhn dataset- Test Loss: 1.1947, Accuracy: 30.28
model_copycat in combined dataset- Test Loss: 2.2964, Accuracy: 30.54


In [24]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms8.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.0973, Accuracy: 27.22
model_copycat in svhn dataset- Test Loss: 1.2072, Accuracy: 47.80
model_copycat in combined dataset- Test Loss: 2.2869, Accuracy: 42.09


In [25]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms9.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.2173, Accuracy: 24.60
model_copycat in svhn dataset- Test Loss: 1.1454, Accuracy: 42.37
model_copycat in combined dataset- Test Loss: 2.2755, Accuracy: 37.44


In [29]:
test_model_10l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms10.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.5759, Accuracy: 32.38
model_copycat in svhn dataset- Test Loss: 22.0764, Accuracy: 19.51
model_copycat in combined dataset- Test Loss: 17.4970, Accuracy: 23.08


In [30]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_ms10.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 5.5759, Accuracy: 32.38
model_copycat in svhn dataset- Test Loss: 1.3212, Accuracy: 3.03
model_copycat in combined dataset- Test Loss: 2.5020, Accuracy: 11.18


## Soft train

In [38]:
test_dataset_mnist

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [39]:
test_dataset_svhn

Dataset SVHN
    Number of datapoints: 26032
    Root location: data/SVHN
    Split: test
    StandardTransform
Transform: Compose(
               Resize(size=(32, 32), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )
Target transform: <function <lambda> at 0x7fd2e295b240>

### MNIST

In [33]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_m1_soft.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.1606, Accuracy: 72.72
model_copycat in svhn dataset- Test Loss: 3.0745, Accuracy: 0.00
model_copycat in combined dataset- Test Loss: 2.8209, Accuracy: 20.18


In [34]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_m2_soft.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.0432, Accuracy: 82.69
model_copycat in svhn dataset- Test Loss: 4.1728, Accuracy: 0.00
model_copycat in combined dataset- Test Loss: 3.5818, Accuracy: 22.95


In [45]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_m3_soft.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 1.9573, Accuracy: 93.29
model_copycat in svhn dataset- Test Loss: 3.1257, Accuracy: 0.00
model_copycat in combined dataset- Test Loss: 2.8014, Accuracy: 25.89


In [46]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_m4_soft.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 1.9511, Accuracy: 96.10
model_copycat in svhn dataset- Test Loss: 3.0848, Accuracy: 0.00
model_copycat in combined dataset- Test Loss: 2.7702, Accuracy: 26.67


In [48]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_m5_soft.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 1.9041, Accuracy: 98.01
model_copycat in svhn dataset- Test Loss: 3.0741, Accuracy: 0.00
model_copycat in combined dataset- Test Loss: 2.7494, Accuracy: 27.20


### MNIST+SVHN

In [13]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms1.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.6205, Accuracy: 53.57
model_copycat in svhn dataset- Test Loss: 2.7473, Accuracy: 24.49
model_copycat in combined dataset- Test Loss: 2.7121, Accuracy: 32.56


In [14]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms2.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.5890, Accuracy: 62.98
model_copycat in svhn dataset- Test Loss: 2.6641, Accuracy: 32.85
model_copycat in combined dataset- Test Loss: 2.6433, Accuracy: 41.21


In [15]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms3.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.8147, Accuracy: 38.93
model_copycat in svhn dataset- Test Loss: 2.6621, Accuracy: 23.47
model_copycat in combined dataset- Test Loss: 2.7045, Accuracy: 27.76


In [16]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms4.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.5436, Accuracy: 66.51
model_copycat in svhn dataset- Test Loss: 2.7687, Accuracy: 45.37
model_copycat in combined dataset- Test Loss: 2.7062, Accuracy: 51.24


In [17]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms5.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.5348, Accuracy: 63.58
model_copycat in svhn dataset- Test Loss: 2.5862, Accuracy: 45.71
model_copycat in combined dataset- Test Loss: 2.5719, Accuracy: 50.67


In [18]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms6.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.4897, Accuracy: 75.61
model_copycat in svhn dataset- Test Loss: 2.6038, Accuracy: 42.34
model_copycat in combined dataset- Test Loss: 2.5722, Accuracy: 51.57


In [19]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms7.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.4061, Accuracy: 71.47
model_copycat in svhn dataset- Test Loss: 2.5958, Accuracy: 52.29
model_copycat in combined dataset- Test Loss: 2.5431, Accuracy: 57.61


In [12]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms8.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.4160, Accuracy: 74.78
model_copycat in svhn dataset- Test Loss: 2.6780, Accuracy: 36.02
model_copycat in combined dataset- Test Loss: 2.6053, Accuracy: 46.78


In [20]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms9.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.5405, Accuracy: 69.53
model_copycat in svhn dataset- Test Loss: 2.5922, Accuracy: 54.59
model_copycat in combined dataset- Test Loss: 2.5779, Accuracy: 58.73


In [22]:
test_model_20l_all_datasets(utils.create_resnet_model('./pths/0_19/copycat_soft_ms10.pth', out_features=20))

model_copycat in mnist dataset- Test Loss: 2.5102, Accuracy: 73.85
model_copycat in svhn dataset- Test Loss: 2.5954, Accuracy: 46.11
model_copycat in combined dataset- Test Loss: 2.5718, Accuracy: 53.81
